# Chesser

### Initial Setup

In [1]:
# Imports
import os, warnings
import matplotlib.pyplot as plt
from matplotlib import gridspec
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
import numpy as np
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from pathlib import Path

# Set Matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('image', cmap='magma')
warnings.filterwarnings("ignore") # to clean up output cells

# Reproducibility
def set_seed(seed=31415):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
set_seed()

# Check Tensorflow identifies GPU and enable memory growth
#physical_devices = tf.config.experimental.list_physical_Devices('GPU')
#print("Num GPUs available: " , len(physical_devices))
#tf.config.experimental.set_memory_grows(physical_devices[0], True)


In [2]:
data_dir = Path('input/classes')
image_count = len(list(data_dir.glob('*/*')))
print(image_count)

1143


In [3]:
batch_size = 64

# Train Dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  seed=123,
  subset="training",
  label_mode='categorical',
  #image_size=(img_height, img_width),
  batch_size=batch_size)

# Validation Dataset
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  seed=123,
  subset="validation",
  label_mode='categorical',
  #image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names
num_classes = len(train_ds.class_names)
print(num_classes, class_names)

Found 1127 files belonging to 13 classes.
Using 902 files for training.
Found 1127 files belonging to 13 classes.
Using 225 files for validation.
13 ['BB', 'BK', 'BN', 'BP', 'BQ', 'BR', 'Empty', 'WB', 'WK', 'WN', 'WP', 'WQ', 'WR']


### Standardize the data

In [4]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.05557754 0.811106


### Configure the dataset for performance

In [5]:
#AUTOTUNE = tf.data.AUTOTUNE

#train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
#val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
#def configure_for_performance(ds):
#  ds = ds.cache()
#  ds = ds.shuffle(buffer_size=1000)
#  ds = ds.batch(batch_size)
#  ds = ds.prefetch(buffer_size=AUTOTUNE)
#  return ds

#train_ds = configure_for_performance(train_ds)
#val_ds = configure_for_performance(val_ds)

### Train model

In [6]:
import os
import numpy as np

from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from keras.constraints import max_norm

# architecture of the neural network
model = Sequential()
model.add(Conv2D(32, kernel_size=(5,5),
                activation='relu',
                kernel_constraint=max_norm(3.)))
model.add(MaxPooling2D(pool_size=(2,2)))
#Conv block 2
model.add(Conv2D(64, kernel_size=(5,5), activation='relu', kernel_constraint=max_norm(3.)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

# Identification
model.add(Flatten())
model.add(Dense(128, activation='relu', name='features'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(13, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
             optimizer='Adam',
             metrics=['accuracy'])
# the loss is minimized by the neural net
# optimizer is the method for training (Adam= stochastic gradient descent, adaptive estimation of first-order and second-order moments)
# metrics accuracy divides total by count, the number of correctly identified images to total number of images

LOGDIR = "./logs"
my_tensorboard = TensorBoard(log_dir = LOGDIR,
                            histogram_freq=1,
                            write_graph=True,
                            write_images=True)
# Hyperparamter
my_epochs = 75

history = model.fit(train_ds,
                   batch_size=batch_size,
                   callbacks=[my_tensorboard],
                   epochs=my_epochs,
                   verbose=1,
                   validation_data=val_ds)

model.save('network.h5')

Epoch 1/75
15/15 [==============================] - 24s 1s/step - loss: 18.0511 - accuracy: 0.0765 - val_loss: 0.4014 - val_accuracy: 0.1644
Epoch 2/75
15/15 [==============================] - 22s 1s/step - loss: 0.3905 - accuracy: 0.1530 - val_loss: 0.2982 - val_accuracy: 0.1733
Epoch 3/75
15/15 [==============================] - 22s 1s/step - loss: 0.3167 - accuracy: 0.2062 - val_loss: 0.3051 - val_accuracy: 0.1956
Epoch 4/75
15/15 [==============================] - 21s 1s/step - loss: 0.2892 - accuracy: 0.2694 - val_loss: 0.3180 - val_accuracy: 0.2178
Epoch 5/75
15/15 [==============================] - 21s 1s/step - loss: 0.2914 - accuracy: 0.3348 - val_loss: 0.3011 - val_accuracy: 0.2533
Epoch 6/75
15/15 [==============================] - 23s 1s/step - loss: 0.2319 - accuracy: 0.4656 